In [1]:
import numpy as np
import json

In [27]:
# Set credentials
# gcloud auth application-default login

# Vertex parameters

In [1]:
from utils.doc_to_vertex_search import get_me_parameters
file_path = "me_parameters.json"
parameters =  get_me_parameters(file_path)

In [2]:
PROJECT_ID = parameters['PROJECT_ID']
LOCATION = parameters['LOCATION']
CHATBOT_NAME = parameters['CHATBOT_NAME']
ME_INDEX_ID = parameters['ME_INDEX_ID']
ME_INDEX_ENDPOINT_ID = parameters['ME_INDEX_ENDPOINT_ID']
ME_INDEX_NAME = parameters['ME_INDEX_NAME']
ME_EMBEDDING_DIR = parameters['ME_EMBEDDING_DIR']
ME_DIMENSIONS = parameters['ME_DIMENSIONS']

In [34]:
PROJECT_ID

'ecg-ai-416210'

In [35]:
LOCATION

'europe-west1'

# Initiate Embeddings, vector store and retriever

In [3]:
# initiate embeddings
from utils.custom_vertexai_embeddings import CustomVertexAIEmbeddings
# Embeddings API integrated with langChain
EMBEDDING_QPM = 100
EMBEDDING_NUM_BATCH = 5
embeddings = CustomVertexAIEmbeddings(
    requests_per_minute=EMBEDDING_QPM,
    num_instances_per_batch=EMBEDDING_NUM_BATCH,
)

2024-04-04 09:52:31.758833: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Model_name will become a required arg for VertexAIEmbeddings starting from Feb-01-2024. Currently the default is set to textembedding-gecko@001


In [4]:
# initiate vector store
from utils.matching_engine import MatchingEngine
me = MatchingEngine.from_components(
    project_id=PROJECT_ID,
    region=LOCATION,
    gcs_bucket_name=f"gs://{ME_EMBEDDING_DIR}".split("/")[2],
    embedding=embeddings,
    index_id=ME_INDEX_ID,
    endpoint_id=ME_INDEX_ENDPOINT_ID,
    # credentials_path=credentials_path
)

In [6]:
# # Test whether search from vector store is working
# me.similarity_search("moyens de réserver un séjour", k=4)

In [5]:
# Expose index to the retriever
NUMBER_OF_RESULTS = 2
SEARCH_DISTANCE_THRESHOLD = 0.6

retriever = me.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": NUMBER_OF_RESULTS,
        "search_distance": SEARCH_DISTANCE_THRESHOLD,
    },
    filters=None,
)

In [8]:
# qn = "What're the methods to reserve a stay?"

In [9]:
# retriever.get_relevant_documents(qn)

# Initiate llm

https://python.langchain.com/docs/integrations/llms/google_vertex_ai_palm

https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/text-chat

## vertex llm

In [19]:
from langchain.llms import VertexAI
llm_vta = VertexAI(
    model_name="text-bison@002", # "chat-bison-32k", "chat-bison@001"
    max_output_tokens=1024,
    temperature=0.2,
    top_p=0.8,
    top_k=40,
    verbose=True,
    streaming = True
)

In [7]:
# #https://python.langchain.com/docs/integrations/llms/google_vertex_ai_palm
# from langchain_google_vertexai import VertexAIModelGarden
# VERTEX_GARDEN_ENDPOINT_ID = "5163099895850074112"
# llm_vertex_garden = VertexAIModelGarden(project=PROJECT_ID , endpoint_id=VERTEX_GARDEN_ENDPOINT_ID,location=LOCATION)

In [22]:
# from vertexai.preview.language_models import ChatModel
# chat_model = ChatModel.from_pretrained("chat-bison@001")


# Initiate template

In [18]:
# from langchain.prompts import PromptTemplate
# template = """SYSTEM: You are an intelligent assistant helping the users with their questions on papers related to laws conditions and always cite the article that you're referring to.
# Please alway reply at the same language that question uses.

# Question: {question}

# Strictly Use ONLY the following pieces of context to answer the question at the end. Think step-by-step and then answer.

# Do not try to make up an answer:
#  - If the answer to the question cannot be determined from the context alone, say "I cannot determine the answer to that."
#  - If the context is empty, just say "I do not know the answer to that."

# =============
# {context}
# =============

# Question: {question}
# Helpful Answer:"""
# QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [27]:
# # Build prompt
# from langchain.prompts import PromptTemplate
# template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer, and ask user to try to ask in another way. Use three sentences maximum. Keep the answer as concise as possible.  
# {context}
# Question: {question}
# Helpful Answer:"""
# QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)

In [90]:
#ref: https://github.com/ArmaanSeth/ChatPDF/blob/main/app.py
from langchain.prompts import PromptTemplate
custom_template = """Given the following conversation and a follow up question, if the follow up question is a question and is relevant to chat history, rephrase the follow up question to be a standalone question, in its original language. 
If the follow up question is not a question or not relevant to chat history, please keep original question as Standalone question instead.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""

QA_CHAIN_PROMPT = PromptTemplate.from_template(custom_template)

#If the follow up question is not a question or not relevant to previous conversation, please keep original question as Standalone question instead.

# Initiate memory

In [91]:
# Set memory
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key=f"chat_history",
    return_messages=True,
    output_key='answer'
)

# Set chatbot

In [92]:
from langchain.chains import ConversationalRetrievalChain

In [15]:
# from langchain.prompts import PromptTemplate
# template = """Given the following conversation and a follow-up question, rephrase the follow-up question to be a standalone question, in its original English.
#                         Chat History:
#                         {chat_history}
#                         Follow-Up Input: {question}
#                         Standalone question:"""
# CUSTOM_QUESTION_PROMPT = PromptTemplate(input_variables=["question", "chat_history"],template=template)

In [93]:
qa = ConversationalRetrievalChain.from_llm(
        llm=llm_vta, #llm_vta, 
        retriever=retriever,
        memory=memory,
        condense_question_prompt=QA_CHAIN_PROMPT ,
        # combine_docs_chain_kwargs={"prompt": QA_CHAIN_PROMPT}, #QA_CHAIN_PROMPT 
        return_source_documents=True,
        return_generated_question=True #to solve error
    )

# Test

In [94]:
memory.clear()

In [95]:
qa({"question": "What are the methods to reserve a stay?"})

Waiting


{'question': 'What are the methods to reserve a stay?',
 'chat_history': [HumanMessage(content='What are the methods to reserve a stay?'),
  AIMessage(content=' The four ways to reserve a stay are:\n- By phone\n- Online\n- By email\n- On-site (only for Homair Vacances campsites)')],
 'answer': ' The four ways to reserve a stay are:\n- By phone\n- Online\n- By email\n- On-site (only for Homair Vacances campsites)',
 'source_documents': [Document(page_content='.  \n \nModalités de Réservation  Il existe quatre moyens de réserver un séjour :  \n \n• -Par téléphone  : auprès de nos équipes de réservation et via les numéros suivants (coût d’un appel \nlocal) :  \n• pour Homair Vacances  au 04.84.39.08.60  \n• pour Tohapi au 04.48.20.20.20  ou +33 4 48 20 20 20 (depuis l’étranger)  \n \n• -Par Internet  : sur les Sites internet de la Société.  \n \n• -Par mail  : via les formulaire s de contact suivants  : \n• pour Homair Vacances  : https://contact.homair.com/hc/fr/requests/new  \n• pour To

In [96]:
qa({"question": "can you reply above in french?"})

Waiting


{'question': 'can you reply above in french?',
 'chat_history': [HumanMessage(content='What are the methods to reserve a stay?'),
  AIMessage(content=' The four ways to reserve a stay are:\n- By phone\n- Online\n- By email\n- On-site (only for Homair Vacances campsites)'),
  HumanMessage(content='can you reply above in french?'),
  AIMessage(content=" ARTICLE 3 - LOI APPLICABLE ET JURIDICTION COMPÉTENTE\n\nLes présentes Conditions Générales et plus généralement le Contrat de location sont régis par la loi française.\n\nLes présentes Conditions Générales ont été rédigées en langue française puis traduites dans différentes versions linguistiques. Pour leur interprétation en cas de litige, seul le texte en langue française fait foi, les versions traduites étant produites à titre informatif.\n\nDans l'hypothèse où l'une des clauses des présentes Conditions Générales serait déclarée nulle et non avenue cela ne saurait en aucun cas affecter la validité et le respect des autres clauses qui de

In [50]:
# memory.load_memory_variables({})

In [97]:
qa({"question": "can you give more details?"})

Waiting


{'question': 'can you give more details?',
 'chat_history': [HumanMessage(content='What are the methods to reserve a stay?'),
  AIMessage(content=' The four ways to reserve a stay are:\n- By phone\n- Online\n- By email\n- On-site (only for Homair Vacances campsites)'),
  HumanMessage(content='can you reply above in french?'),
  AIMessage(content=" ARTICLE 3 - LOI APPLICABLE ET JURIDICTION COMPÉTENTE\n\nLes présentes Conditions Générales et plus généralement le Contrat de location sont régis par la loi française.\n\nLes présentes Conditions Générales ont été rédigées en langue française puis traduites dans différentes versions linguistiques. Pour leur interprétation en cas de litige, seul le texte en langue française fait foi, les versions traduites étant produites à titre informatif.\n\nDans l'hypothèse où l'une des clauses des présentes Conditions Générales serait déclarée nulle et non avenue cela ne saurait en aucun cas affecter la validité et le respect des autres clauses qui demeur

In [98]:
qa({"question": "can i bring a dog?"})

Waiting


{'question': 'can i bring a dog?',
 'chat_history': [HumanMessage(content='What are the methods to reserve a stay?'),
  AIMessage(content=' The four ways to reserve a stay are:\n- By phone\n- Online\n- By email\n- On-site (only for Homair Vacances campsites)'),
  HumanMessage(content='can you reply above in french?'),
  AIMessage(content=" ARTICLE 3 - LOI APPLICABLE ET JURIDICTION COMPÉTENTE\n\nLes présentes Conditions Générales et plus généralement le Contrat de location sont régis par la loi française.\n\nLes présentes Conditions Générales ont été rédigées en langue française puis traduites dans différentes versions linguistiques. Pour leur interprétation en cas de litige, seul le texte en langue française fait foi, les versions traduites étant produites à titre informatif.\n\nDans l'hypothèse où l'une des clauses des présentes Conditions Générales serait déclarée nulle et non avenue cela ne saurait en aucun cas affecter la validité et le respect des autres clauses qui demeureront pl

In [69]:
qa({"question": "all types of dog?"})

Waiting


{'question': 'all types of dog?',
 'chat_history': [HumanMessage(content='What are the methods to reserve a stay?'),
  AIMessage(content=' The four ways to reserve a stay are:\n- By phone\n- Online\n- By email\n- On-site (only for Homair Vacances campsites)'),
  HumanMessage(content='can you reply above in french?'),
  AIMessage(content=' Il existe quatre moyens de réserver un séjour\xa0:  \n \n• Par téléphone  \n• Par Internet  \n• Par mail  \n• Sur place'),
  HumanMessage(content='can you give more details?'),
  AIMessage(content=' Le contexte ne fournit pas de détails supplémentaires.'),
  HumanMessage(content='can i bring a dog?'),
  AIMessage(content=' Oui, vous pouvez amener un chien, mais il doit être de catégorie 3 ou 4 et le camping doit accepter les chiens.'),
  HumanMessage(content='all types of dog?'),
  AIMessage(content=' Non, seuls les chiens de catégorie 1 et 2 sont interdits.')],
 'answer': ' Non, seuls les chiens de catégorie 1 et 2 sont interdits.',
 'source_document

In [70]:
qa({"question": "what if it's red"})

Waiting


{'question': "what if it's red",
 'chat_history': [HumanMessage(content='What are the methods to reserve a stay?'),
  AIMessage(content=' The four ways to reserve a stay are:\n- By phone\n- Online\n- By email\n- On-site (only for Homair Vacances campsites)'),
  HumanMessage(content='can you reply above in french?'),
  AIMessage(content=' Il existe quatre moyens de réserver un séjour\xa0:  \n \n• Par téléphone  \n• Par Internet  \n• Par mail  \n• Sur place'),
  HumanMessage(content='can you give more details?'),
  AIMessage(content=' Le contexte ne fournit pas de détails supplémentaires.'),
  HumanMessage(content='can i bring a dog?'),
  AIMessage(content=' Oui, vous pouvez amener un chien, mais il doit être de catégorie 3 ou 4 et le camping doit accepter les chiens.'),
  HumanMessage(content='all types of dog?'),
  AIMessage(content=' Non, seuls les chiens de catégorie 1 et 2 sont interdits.'),
  HumanMessage(content="what if it's red"),
  AIMessage(content=" I don't know.")],
 'answer